# Similar Qs

Testing out Python PDF Capabilities to tokenize RFPs

In [1]:
import os
from pinecone import Pinecone
import pandas as pd
import numpy as np
import ast
import tiktoken
from openai import OpenAI
from tqdm.autonotebook import tqdm

import concurrent.futures
from concurrent.futures import ThreadPoolExecutor

client = OpenAI()

# embedding model parameters
embedding_model = "text-embedding-3-large"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

encoding = tiktoken.get_encoding(embedding_encoding)

#turns off SettingWithoutCopy warning
pd.options.mode.chained_assignment = None

c:\Users\E707254\AppData\Local\miniconda3\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
pd.DataFrame([[1,2], [3, None]]).fillna("")

,0,1
0,1,2.0
1,3,


In [10]:
pd.read_csv('new_q_a_TEMPLATE.csv')

,question,answer,rfp,project,tech,general,state,iso/region,month,year
0,What is the project name?,Las Camas Solar Park,cpa_rfo_2024,Las Camas,Solar,True,CA,CAISO,7,2024


In [3]:
pc = Pinecone(api_key=os.environ['PINECONE_API_KEY'])
index = pc.Index('rfp-response-qs')

In [33]:
rfp_name = 'sdcp_c15_rfp_2024'
field = 'project'
original = 'Brooks Solar and Storage'
new = 'Brooks'

In [34]:
ids_to_change = [match.id for match in index.query(id=f'{rfp_name}_0', top_k=200, namespace='rfp-response', filter={'$and': [{field: original}, {'rfp': rfp_name}]})['matches']]

In [37]:
len(ids_to_change)

90

In [38]:
for id in ids_to_change:
    index.update(id=id, set_metadata={'project': new}, namespace='rfp-response', async_req=True)

In [5]:
list(index.list(prefix='sdcp_c15_rfp_2024', namespace='rfp-response'))

[['sdcp_c15_rfp_2024_0',
  'sdcp_c15_rfp_2024_1',
  'sdcp_c15_rfp_2024_10',
  'sdcp_c15_rfp_2024_100',
  'sdcp_c15_rfp_2024_101',
  'sdcp_c15_rfp_2024_102',
  'sdcp_c15_rfp_2024_103',
  'sdcp_c15_rfp_2024_104',
  'sdcp_c15_rfp_2024_105',
  'sdcp_c15_rfp_2024_106',
  'sdcp_c15_rfp_2024_107',
  'sdcp_c15_rfp_2024_108',
  'sdcp_c15_rfp_2024_109',
  'sdcp_c15_rfp_2024_11',
  'sdcp_c15_rfp_2024_110',
  'sdcp_c15_rfp_2024_111',
  'sdcp_c15_rfp_2024_112',
  'sdcp_c15_rfp_2024_113',
  'sdcp_c15_rfp_2024_114',
  'sdcp_c15_rfp_2024_115',
  'sdcp_c15_rfp_2024_116',
  'sdcp_c15_rfp_2024_117',
  'sdcp_c15_rfp_2024_118',
  'sdcp_c15_rfp_2024_119',
  'sdcp_c15_rfp_2024_12',
  'sdcp_c15_rfp_2024_120',
  'sdcp_c15_rfp_2024_121',
  'sdcp_c15_rfp_2024_122',
  'sdcp_c15_rfp_2024_123',
  'sdcp_c15_rfp_2024_124',
  'sdcp_c15_rfp_2024_125',
  'sdcp_c15_rfp_2024_126',
  'sdcp_c15_rfp_2024_127',
  'sdcp_c15_rfp_2024_128',
  'sdcp_c15_rfp_2024_129',
  'sdcp_c15_rfp_2024_13',
  'sdcp_c15_rfp_2024_130',
  'sdcp_c

In [3]:
import rfp_q_sim as rfp

pd.DataFrame([[key, ''] for key in rfp.filter_cat_dict.keys()], columns=['Filter', 'Selection'])

,Filter,Selection
0,Project Name,
1,ISO/Region,
2,State,
3,Technology,
4,Include General (not project/RFP specific) que...,
5,Specific RFP,


In [4]:
def get_embedding(text, engine=embedding_model):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=engine).data[0].embedding

In [40]:
c = pd.DataFrame([[key, '4'] for key in rfp.filter_cat_dict.keys()], columns=['Filter', 'Selection'])
c.iloc[0, 1] = ''
c[c['Selection'] != '']

,Filter,Selection
1,ISO/Region,4
2,State,4
3,Technology,4
4,Include General (not project/RFP specific) que...,4
5,Specific RFP,4


In [30]:
rfp_months = {'aps_rfp_2023': '8',
 'cea_offer_2024': '7',
 'cpa_rfo_2024': '7',
 'cpa_rfp_2023': '7',
 'csu_rfp_2023': '1',
 'evergy_rfp_2023': '3',
 'gcpud_rfp_2024': '2',
 'pso_rfp_2023': '1',
 'sce_rfp_2023': '12',
 'srp_rfp_2023': '4',
 'svce_rfp_2023': '12',
 'tid_rfp_2024': '6'}

In [31]:
all_qs = index.query(top_k=10000, vector=get_embedding("hello world"), namespace="rfp-response", include_metadata=True)['matches']

for q in all_qs:
    index.update(id = q['id'], namespace = 'rfp-response', set_metadata = {'month': rfp_months[q['metadata']['rfp']]})

In [23]:
pinecone_metadata = [item.metadata for item in index.query(top_k=10000, vector=get_embedding("hello world"), namespace="rfp-response", include_metadata=True)['matches']]
[item.pop('answer') for item in pinecone_metadata]
[item.pop('question') for item in pinecone_metadata]
pinecone_metadata_unique = {key: set([item[key] for item in pinecone_metadata]) for key in pinecone_metadata[0].keys()}

In [28]:
pinecone_metadata_unique['rfp']

{'aps_rfp_2023',
 'cea_offer_2024',
 'cpa_rfo_2024',
 'cpa_rfp_2023',
 'csu_rfp_2023',
 'evergy_rfp_2023',
 'gcpud_rfp_2024',
 'pso_rfp_2023',
 'sce_rfp_2023',
 'srp_rfp_2023',
 'svce_rfp_2023',
 'tid_rfp_2024'}

In [24]:
pinecone_metadata_unique

{'general': {False, True},
 'iso/region': {'CAISO', 'SPP', 'WECC'},
 'project': {'Avondale',
  'Casa Grande',
  'Crossing Trails II',
  'Flatlands',
  'Las Camas',
  'Plum Nellie',
  'Sonrisa',
  'Western Star'},
 'rfp': {'aps_rfp_2023',
  'cea_offer_2024',
  'cpa_rfo_2024',
  'cpa_rfp_2023',
  'csu_rfp_2023',
  'evergy_rfp_2023',
  'gcpud_rfp_2024',
  'pso_rfp_2023',
  'sce_rfp_2023',
  'srp_rfp_2023',
  'svce_rfp_2023',
  'tid_rfp_2024'},
 'state': {'AZ', 'CA', 'CO', 'KS', 'WA'},
 'tech': {'BESS', 'Solar', 'Wind'},
 'year': {'2023', '2024'}}

In [8]:
pd.DataFrame([[key, list(pinecone_metadata_unique[rfp.filter_cat_dict[key]])] for key in rfp.filter_cat_dict.keys()], columns=['Filter', 'Selection'])

,Filter,Selection
0,Project Name,"[Plum Nellie, Casa Grande, Crossing Trails II,..."
1,ISO/Region,"[SPP, CAISO, WECC]"
2,State,"[WA, CA, KS, CO, AZ]"
3,Technology,"[BESS, Wind, Solar]"
4,Include General (not project/RFP specific) que...,"[False, True]"
5,Specific RFP,"[aps_rfp_2023, svce_rfp_2023, cpa_rfp_2023, cp..."


In [13]:
list(new_csv.columns)

['question',
 'answer',
 'rfp',
 'project',
 'tech',
 'general',
 'state',
 'iso/region']

In [18]:
new_csv = pd.read_csv('new_q_a_bank.csv').dropna()

def check_new_csv(csv):
    assert list(csv.columns) == [
        'question', 'answer', 'rfp', 'project', 'tech', 'general', 'state', 'iso/region', 'month', 'year'
    ], 'Columns must be question, answer, rfp, project, tech, general, state, iso/region, in that order'

check_new_csv(new_csv)

In [ ]:
pinecone_metadata_unique

In [ ]:
pinecone_metadata

In [ ]:

#takes in a dataframe with at least an 'id' and 'text' column
def parallel_embedding_for_df(chunks_df, embed_function):

    section_data = list(zip(chunks_df['id'], chunks_df['question']))

    section_data_with_embeddings = []

    with ThreadPoolExecutor() as executor:
        
        future_to_section = {executor.submit(embed_function, section[1]): section for section in section_data}

        for future in concurrent.futures.as_completed(future_to_section):
            section = future_to_section[future]

            try:
                embedding = future.result()
                section_data_with_embeddings.append([section[0], embedding])
            except Exception as exc:
                print(f"{section[0]} generated an exception: {exc}")
            
    return pd.DataFrame(section_data_with_embeddings, columns=['id', 'embedding'])

def prepare_df_for_pinecone(df, id_column='id', embedding_column='embedding'):
        return [{'id': row[id_column], 'values': row[embedding_column], 'metadata': {f'{column}': row[column] for column in df.columns if column not in [
                id_column, embedding_column]}} for _, row in df.iterrows()]

def cut_up_list(input_list, chunk_size):
        """Yield successive chunks of chunk_size from input_list."""
        for i in range(0, len(input_list), chunk_size):
            yield input_list[i:i + chunk_size]

# def cosine_similarity(a, b):
#     return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [ ]:
# function that takes in a new csv of questions, answers, and metadata and uploads it
def upload_new_qs(new_q_a_csv):

    new_q_a_csv = new_q_a_csv.dropna(subset=['question', 'answer']).reset_index(drop=True)

    new_q_a_csv['counter'] = new_q_a_csv.groupby('rfp').cumcount()

    # Add the 'id' column by combining 'rfp' and 'counter'
    new_q_a_csv['id'] = new_q_a_csv['rfp'] + '_' + new_q_a_csv['counter'].astype(str)

    # Drop the intermediate 'counter' column if not needed
    new_q_a_csv = new_q_a_csv.drop('counter', axis=1)

    ids_embeddings = parallel_embedding_for_df(new_q_a_csv, get_embedding)

    new_q_a_csv = new_q_a_csv.merge(ids_embeddings, on='id', how='inner')

    q_a_bank_pc = prepare_df_for_pinecone(new_q_a_csv)

    for sub_list in tqdm(list(cut_up_list(q_a_bank_pc, 30))):
        index.upsert(vectors=sub_list, namespace='rfp-response', async_req=True)

In [ ]:
new_q_bank = pd.read_csv("new_q_a_bank.csv")

upload_new_qs(new_q_bank)

In [ ]:
def delete_rfp_from_pinecone_namespace(rfp_name, index=index, namespace='rfp-response'):
    index.delete(list(index.list(prefix=rfp_name, namespace=namespace))[0], namespace=namespace)

In [ ]:
def get_suggestions(question, index=index, k=5, filter_dict = {}, print_filter_dict=False):
    question_embedding = get_embedding(question)

    confirmed_filter_dict = dict()

    confirmed_filter_dict = {key: value if key in ['rfp_name', 'general', 'project', 'isoregion', 'tech'] else print(f'{key} is not filterable')for key, value in filter_dict.items()}

    if print_filter_dict:
        print(confirmed_filter_dict)
        
    results = index.query(vector=question_embedding, top_k=k, namespace='rfp-response', include_metadata=True, filter=confirmed_filter_dict)

    q_a = [[match['metadata']['question'], match['metadata']['answer'], match['metadata']['rfp'], match['metadata']['project']] for match in results['matches']]

    if not len(q_a):
        return pd.DataFrame(columns=['question', 'answer', 'rfp_name', 'project'])

    if q_a[0][0] == question:

        q_a = [q_a[0]] + [['', ' ', ' ']] * 4

    return pd.DataFrame(q_a, columns=['question', 'answer', 'rfp_name', 'project'])

In [ ]:
def run_similarities_on_q_df(q_df, filter_dict={}):

    # Create a list to store the suggestions
    suggestions_list = []

    # iterate through the questions in the dataframe
    for question in q_df['question']:

        # Create a dictionary to store the question and the suggestions
        suggestion_row = {'question_actual': question}

        # Get the suggestions for the question using the get_suggestions function
        suggestions = [{f'{inner_key}_{outer_key}': inner_value for inner_key, inner_value in outer_value.items()} for outer_key, outer_value in 
                    get_suggestions(question, filter_dict = filter_dict, print_filter_dict=False).to_dict(orient='index').items()]

        # Update the suggestion_row dictionary with the suggestions
        for sugggestion in suggestions:
            suggestion_row.update(sugggestion)

        # Append the suggestion_row to the suggestions_list
        suggestions_list.append(suggestion_row)
        
    # Return the suggestions_list as a dataframe
    return pd.DataFrame(suggestions_list)

In [ ]:
get_suggestions("Pnoda", filter_dict={'project': 'Sonrisa'})

In [ ]:
import re

def to_snake_case(text):
    # Replace all non-alphanumeric characters with a space
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    # Replace uppercase letters with lowercase and precede them with an underscore if they are not at the start
    text = re.sub(r'(?<!^)(?=[A-Z])', '_', text).lower()
    # Replace spaces and underscores with a single underscore
    text = re.sub(r'[\s_]+', '_', text)
    return text

In [ ]:
import pandas as pd

# Load the .xlsm file
file_path = 'Attachment_C_Offer_Form_03312023_ia5ufs.xlsm'  # Replace with your file path

def save_sheets_as_csv(file_path):
    # Load the Excel file

    excel_file = pd.ExcelFile(file_path, engine='openpyxl')

    # Loop through each sheet in the file
    for sheet_name in excel_file.sheet_names:
        # Load the sheet into a DataFrame
        df = pd.read_excel(excel_file, sheet_name=sheet_name)
        
        # Create a CSV file name based on the sheet name
        csv_file_name = f"{to_snake_case(sheet_name)}.csv"
        
        # Save the DataFrame as a CSV file with UTF-8 encoding
        df.to_csv(f'rfp_sheets/{csv_file_name}', encoding='utf-8', index=False)

    print("All sheets have been saved as CSV files.")

In [ ]:
q_a_suggestions_df.to_csv('q_a_suggestions.csv', index=False)

In [ ]:
ava_qs = pd.read_csv('rfp_sheets/ava_rfp_qs.csv')

In [ ]:
ava_qs_suggestions = run_similarities_on_q_df(ava_qs, filter_dict={'project': 'Las Camas'})

In [ ]:
ava_qs_suggestions.to_csv('rfp_sheets/ava_qs_suggestions.csv', index=False)

In [ ]:
sce = '../RFPs/SCE/'
svce = '../RFPs/Completed/SVCE/'

In [ ]:
workbook_qs = pd.read_csv('calchoice_rfp_qs.csv')
#workbook_qs.embedding = workbook_qs.embedding.apply(ast.literal_eval)
q_bank = pd.read_csv(sce + 'sonrisa_q_bank_all.csv')
q_bank.embedding = q_bank.embedding.apply(ast.literal_eval)

In [ ]:
workbook_qs['embedding'] = workbook_qs.question.apply(lambda x: get_embedding(x, engine=embedding_model))
# q_bank['embedding'] = q_bank.question.apply(lambda x: get_embedding(x, engine=embedding_model))

workbook_qs.to_csv('calchoice_rfp_qs.csv', index=False)
# q_bank.to_csv(sce + 'sonrisa_q_bank_all.csv', index=False)

In [ ]:
#workbook_qs.drop(columns = 'embedding').to_csv(svce + 'svce_qs.csv', index = False)

In [ ]:
import heapq

def track_top_n(entries, n):
    heap = []
    
    for number, similarity in entries:
        if len(heap) < n:
            heapq.heappush(heap, (similarity, number))
        else:
            if similarity > heap[0][0]:
                heapq.heappop(heap)
                heapq.heappush(heap, (similarity, number))
    
    # Convert back the similarity scores and return the top 5
    return [(num, sim) for (sim, num) in heap]


def top_n_sim_qs(embedding, bank=q_bank, n=5):
    
    bank['similarity'] = [None] * bank.shape[0]
    
    for index, row in bank.iterrows():
        bank.loc[index, 'similarity'] = cosine_similarity(row.embedding, embedding)
    
    return track_top_n(list(zip(bank.index, bank.similarity)), n=n)  

def flatten_extend(matrix):
    flat_list = []
    for row in matrix:
        flat_list.extend(row)
    return flat_list

def all_new_qs_most_sim(qs=workbook_qs, bank=q_bank, n=5):
    sims = []
    new_cols = flatten_extend([['question%d' % i, 'answer%d' % i, 'rfp%d' % i, 'similarity%d' % i] for i in range(n)])
    
    for index, row in qs.iterrows():
        row_sim_stats = sorted(top_n_sim_qs(row.embedding, n=n), key=lambda x: x[1], reverse=True)
        row_sim_info = flatten_extend([[bank.question[stat[0]], bank.answer[stat[0]], bank.rfp[stat[0]], stat[1]] 
                                      for stat in row_sim_stats])
        
        sims.append(row_sim_info)
        
    return pd.DataFrame(sims, columns=new_cols)

In [ ]:
workbook_qs_with_sims = workbook_qs.join(all_new_qs_most_sim()).drop(columns=['embedding'])

In [ ]:
workbook_qs_with_sims

In [ ]:
workbook_qs_with_sims.to_csv('calchoice_rfp_qs_with_sims.csv', index=False)

In [ ]:
no_embed_q_bank = q_bank.drop(columns=['embedding', 'similarity'])

In [ ]:
no_embed_q_bank.to_csv(sce + 'no_embed_q_bank.csv', index=False)

In [ ]:
pd.read_csv(svce + 'svce_qs.csv')